In [1]:
import os
os.getpid()

21050

In [2]:
from pathlib import Path

import pandas as pd
import numpy as np

from tqdm.auto import tqdm
tqdm.pandas()

In [3]:
df = pd.read_csv('data/NLP_task1/Evaluation-dataset.csv', header=None)

In [4]:
df = df.rename({0: 'review'}, axis=1)
df.sample(2)

,review,1,2,3,4,5,6,7,8,9,10,11,12,13,14
1235,Choosing the right tyres at the right price on...,value for money positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4840,Always good service at ATS.,garage service positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.drop('review', axis=1)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,garage service positive,ease of booking positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,garage service positive,value for money positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,value for money positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ease of booking positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,location positive,value for money positive,ease of booking positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10127,refund positive,delivery punctuality positive,refund timescale positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10128,length of fitting positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10129,location positive,delivery punctuality positive,length of fitting positive,value for money positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
aspects_df = df.drop('review', axis=1)
aspects_df.sample(2)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
4298,garage service positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df['review'].value_counts()

Tires where delivered to the garage of my choice,the garage notified me when they had been delivered. A day and time was arranged with the garage and I went and had them fitted,a Hassel free experience.    1
Easy use of website and a very quick service, would highly recommend                                                                                                                                          1
Very happy with the service. I will use again.                                                                                                                                                                1
The order was fulfilled exactly as promised                                                                                                                                                                   1
Quick and no hassle                                                                                                                                                     

In [11]:
def collate_reviews(rows):
    rows = rows.dropna()
    rows = rows.to_list()
    return rows

In [15]:
aspects = aspects_df.progress_apply(collate_reviews, axis=1)
aspects.sample(2)

  0%|          | 0/10132 [00:00<?, ?it/s]

2445                           [value for money positive]
214     [garage service positive, length of fitting po...
dtype: object

In [18]:
aspects.apply(len).value_counts(dropna=False)

1     3897
2     2705
0     2007
3     1085
4      313
5       91
6       22
7        5
8        4
9        2
14       1
dtype: int64

In [29]:
pd.Series(aspects_df.values.ravel()).value_counts()

value for money positive                4780
garage service positive                 2031
ease of booking positive                1187
location positive                       1063
length of fitting positive               657
                                        ... 
failed payment positive                    1
discount not applied positive              1
change of time positive                    1
mobile fitter didn't arrive positive       1
refund not actioned positive               1
Length: 62, dtype: int64